In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator)
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import pytensor
import pytensor.tensor as pt
from numpy.ma import masked_values
import pickle
import os
import sys
import time

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v5.6.1


In [2]:
# # Extract Treat names from dataset
# Treat_list = pd.read_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Readytofit1.csv')
# Treat_list['tempC1'] = 0
# Treat_list.loc[(Treat_list['Avg_tempC'] > 27.5) & (Treat_list['Avg_tempC'] <= 32.5), 'tempC1'] = 30
# Treat_list.loc[(Treat_list['Avg_tempC'] > 32.5) & (Treat_list['Avg_tempC'] <= 37.5), 'tempC1'] = 35
# Treat_list.loc[(Treat_list['Avg_tempC'] > 37.5) & (Treat_list['Avg_tempC'] <= 42.5), 'tempC1'] = 40

# unique_values = Treat_list[['Treat','Index','tempC1']].drop_duplicates()

# # Save the name list to a csv file
# unique_values.to_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Treat_list1.csv', index=False)

C4_photosynthesis_model

In [3]:
# Calculate Cm by starting with Ci with 10 auto-iterations, Cm is used in AC1 calculation.
def tscanCM(np_Cm, Ci, VcmaxT, VpmaxT, RDT, GMT, tempC):
    Ac1 = fAc1((Ci,np_Cm), VcmaxT, VpmaxT, RDT, tempC)
    np_Cm = Ci - Ac1/GMT
    return np_Cm

def fCm(Ci, VcmaxT, VpmaxT, iterations, RDT, tempC):
    GM25=1.2
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    np_Cm = Ci

    outInit = np_Cm
    results, updates = pytensor.scan(fn=tscanCM,
                                    outputs_info=outInit,
                                    non_sequences=[Ci, VcmaxT, VpmaxT, RDT, GMT, tempC],
                                    n_steps = iterations)
    return results[-1]


def fI2(PARi, alpha_PSII):
    return PARi*alpha_PSII

def fJt(I2, JmaxT, theta):
    return (I2+JmaxT-np.sqrt((I2+JmaxT)**2-4*theta*JmaxT*I2))/(2*theta)

def fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7):
    return Ci*GBST - (1/GMT)*RDT*GBST + (1/GMT)*GBST*x1 + GBST*x1*x10 + RDT*GBST*x12 + GBST*x13 + GBST*x3 - RMT*x8 + Ci*x4*x8 - (1/GMT)*RDT*x4*x8 + (1/GMT)*x1*x4*x8 + x5*x8 - RDT*x6*x8 + x1*x6*x8 - x7*x8

def fd(GBST,GMT,x12,x4,x8,x6):
    return (1/GMT)*GBST - GBST*x12 + (1/GMT)*x4*x8 + x6*x8

def fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11):
    return ((-1/GMT)*GBST - (1/GMT)*x4*x8 + GBST*x12 - x6*x8)*(-RMT*RDT*x8 + RMT*x1*x8 + Ci*RDT*GBST + Ci*RDT*x4*x8 - Ci*GBST*x1 - Ci*x1*x4*x8 + RDT*GBST*x13 + RDT*GBST*x3 + RDT*x5*x8 - RDT*x7*x8 + GBST*x1*x11 - x1*x5*x8 + x1*x7*x8)


def fAc1(data, VcmaxT, VpmaxT, RDT, tempC):
    (Ci, Cm) = data
    GM25=1.2;ALPHA=0.1;GBST=0.003;OM=210000;PHI=2.0;
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KPT=75*pt.exp(36300*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KCT=1210*pt.exp(64200*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KOT=292000*pt.exp(10500*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    VCMAXVOMAXT=5.513*pt.exp(21265*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    GAMMASTART=0.5/(KOT/KCT*VCMAXVOMAXT)
    RMT=0.5*RDT
    x1=VcmaxT;x3=KCT;x4=VpmaxT/(Cm+KPT);x5=0;x6=1;x7=0;x8=1;x10=(GAMMASTART*ALPHA)/0.047/GBST;x11=GAMMASTART*OM;x12=(KCT/KOT)*ALPHA/0.047/GBST;x13=(KCT/KOT)*OM
    A = fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7)
    C = A
    D = fd(GBST,GMT,x12,x4,x8,x6)
    B = fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11)
    Ac1 = (-((A**2 - 4*B)**0.5)+C)/(2*D)
    return Ac1

def fAj(data, theta, alpha_PSII, JmaxT, RDT, tempC):
    (PARi, Ci) = data
    GM25=1.2;ALPHA=0.1;GBST=0.003;OM=210000;PHI=2.0;
    
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KCT=1210*pt.exp(64200*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KOT=292000*pt.exp(10500*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    VCMAXVOMAXT=5.513*pt.exp(21265*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    GAMMASTART=0.5/(KOT/KCT*VCMAXVOMAXT)
    RMT=0.5*RDT
    X=PHI/(3+PHI)
    FCYC=0.25*PHI
    Z=(3-FCYC)/(4*(1-FCYC))
    I2 = fI2(PARi, alpha_PSII)
    Jt = fJt(I2, JmaxT, theta)
    x1=(Z*(1-X)*Jt/3);x3=0;x4=0;x5=(Z*X*Jt/PHI);x6=1;x7=0;x8=1;x10=(GAMMASTART*ALPHA)/0.047/GBST;x11=GAMMASTART*OM;x12=7.0/3.0*GAMMASTART*ALPHA/0.047/GBST;x13=7.0/3.0*GAMMASTART*OM
    A = fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7)
    C = A
    D = fd(GBST,GMT,x12,x4,x8,x6)
    B = fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11)
    Aj = (-((A**2 - 4*B)**0.5)+C)/(2*D)
    return Aj

def fA(Aj,Ac1):
    A = pt.minimum(Ac1, Aj)
    return A

Fitting using normal distributed priors of Alex's with 5% sigma as reference on sorghum

In [4]:
# Calculate Cm by starting with Ci with 10 auto-iterations, Cm is used in AC1 calculation.
data = pd.read_csv("/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Readytofit1.csv", header=0)

# Say which pair of curve we want to fit.
index_values = list(range(1, 126))
# Remove maize and millet curves
remove_values = [1,9,16,19,21,34,38,40,45,48,55,58,62,64,70,71,72,73,74,75,80,81,83,84,85,89,94,95,109,114,123,125]
index_values = [i for i in index_values if i not in remove_values]

# Define the funciton to calculate mode (value with highest density) of estimated coefficients from the posterior distribution
def mode(x):
    hist, bin_edges = np.histogram(x, bins=500)
    mode_index = np.argmax(hist)
    mode_value = (bin_edges[mode_index] + bin_edges[mode_index + 1]) / 2
    return mode_value

# Define the funciton to calculate the median of estimated coefficients from the posterior distribution
def median(x):
    return np.median(x)

# Read in the selected ACI and AI pair
for index_value in index_values:

    filtered_data = data[data['Index'] == index_value]
    obsAnet = filtered_data["A"].values
    PARi = filtered_data["PARi"].values
    Ci = filtered_data["Ci"].values
    tempC = filtered_data["tempC"].values

    # Build Bayesian model
    model_1 = pm.Model()
    with model_1:
        # Define prior
        alpha_PSII = pm.TruncatedNormal('alpha_PSII', mu=0.2048, sigma=0.01,lower=1e-6)
        JmaxT = pm.TruncatedNormal('JmaxT', mu=450.98, sigma=22.549, lower=1e-6)
        RdT = pm.TruncatedNormal('RdT', mu=1.324, sigma=0.0662, lower=1e-6)

        theta = 0.3
        RDT = RdT
        Aj = fAj((PARi, Ci), theta, alpha_PSII, JmaxT, RDT, tempC)

        VcmaxT = pm.TruncatedNormal('VcmaxT', mu=63.884, sigma=3.1942,lower=1e-6)
        VpmaxT = pm.TruncatedNormal('VpmaxT', mu=186.04, sigma=9.302,lower=1e-6)

        # Cm is calculated by using Ci with 10 iterations
        iteration = 10
        Cm = fCm(Ci, VcmaxT, VpmaxT, iteration, RDT, tempC)
        Ac1 = fAc1((Ci, Cm), VcmaxT, VpmaxT, RDT, tempC)

        sigma_Anet = pm.Uniform('sigma_Anet', lower=1e-6, upper=10)

        # Define likilyhood
        Anet = pm.Normal('Anet', mu=fA(Aj, Ac1), sigma=sigma_Anet, observed=obsAnet)

    # Sampling, numbers of draws and tune, target_accept are set based on study needs. Here I set 500 draws with 1500 tune as an example.
    # Draw: The number of samples to be drawn from the posterior distribution after the tuning phase.
    # Tune: The number of samples used for tuning the MCMC algorithm, which are discarded and not used for inference.
    # Target_accept: The target acceptance rate for the proposed samples in the MCMC algorithm.

    idata = pm.sample(model=model_1, draws=5000, tune=15000, target_accept=0.9)

    # Extract and save the draw data from four MCMC sampling chains
    posterior_samples = idata.posterior
    posterior_df = posterior_samples.to_dataframe()

    # Indicating which draw data are from which sampling chain. We have four chains running in the MCMC sampling, the chain index starts from 0
    posterior_df.reset_index(inplace=True)
    posterior_df.rename(columns={'chain': 'Chain'}, inplace=True)

    # Save samples to CSV file
    posterior_df.to_csv(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Sampled_idata_Exp_1/idata_{index_value}.csv", index=False)

    # Plot posterior
    plt.figure(figsize=(12, 8))
    az.plot_trace(idata)
    
    for ax in plt.gcf().axes:
        ax.title.set_fontsize(20)
        for label in ax.get_xticklabels()+ax.get_yticklabels():
            label.set_fontsize(18)

    plt.tight_layout()
    
    # Save plots
    plt.savefig(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Posterior_distribution_Exp_1/posterior_{index_value}.png", dpi=300)
    plt.close()

    # Save the summary of estiamted posterior distribution
    summary_df = az.summary(idata, stat_funcs={'median': np.median,'mode': mode})
    summary_df['Treat_index'] = index_value
    summary_df['Coefficient'] = ['alpha_PSII', 'Jmax', 'Rd', 'Vcmax', 'Vpmax', 'sigma_Anet']
    summary_df.to_csv(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Estimated_coefficients_Exp_1/{index_value}.csv", index=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 360 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 261 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 280 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 198 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


...

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

Fitting using normal distributed priors of Alex's with larger sigma as reference on maize and millet

In [5]:
# Calculate Cm by starting with Ci with 10 auto-iterations, Cm is used in AC1 calculation.
data = pd.read_csv("/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Readytofit1.csv", header=0)

# Say which pair of curve we want to fit.
# Here I used same prior on maize and millet
index_values = [1,9,16,19,21,34,38,40,45,48,55,58,62,64,70,71,72,73,74,75,80,81,83,84,85,89,94,95,109,114,123,125]
# Millet curves: index_values = [9,21,34,40,48,58,64,70,71,72,73,74,75,80,81,83,84,85]


# Define the funciton to calculate mode (value with highest density) of estimated coefficients from the posterior distribution
def mode(x):
    hist, bin_edges = np.histogram(x, bins=500)
    mode_index = np.argmax(hist)
    mode_value = (bin_edges[mode_index] + bin_edges[mode_index + 1]) / 2
    return mode_value

# Define the funciton to calculate the median of estimated coefficients from the posterior distribution
def median(x):
    return np.median(x)

# Read in the selected ACI and AI pair
for index_value in index_values:

    filtered_data = data[data['Index'] == index_value]
    obsAnet = filtered_data["A"].values
    PARi = filtered_data["PARi"].values
    Ci = filtered_data["Ci"].values
    tempC = filtered_data["tempC"].values

    # Build Bayesian model
    model_1 = pm.Model()
    with model_1:
        # Define prior
        alpha_PSII = pm.TruncatedNormal('alpha_PSII', mu=0.256, sigma=0.0128,lower=1e-6)
        JmaxT = pm.TruncatedNormal('JmaxT', mu=411.08, sigma=20.554,lower=1e-6)
        RdT = pm.TruncatedNormal('RdT', mu=2.4923, sigma=0.1246,lower=1e-6)

        theta = 0.3
        RDT = RdT
        Aj = fAj((PARi, Ci), theta, alpha_PSII, JmaxT, RDT, tempC)

        VcmaxT = pm.TruncatedNormal('VcmaxT', mu=57.777, sigma=2.8889,lower=1e-6)
        VpmaxT = pm.TruncatedNormal('VpmaxT', mu=222.94, sigma=11.147,lower=1e-6)

        # Cm is calculated by using Ci with 10 iterations
        iteration = 10
        Cm = fCm(Ci, VcmaxT, VpmaxT, iteration, RDT, tempC)
        Ac1 = fAc1((Ci, Cm), VcmaxT, VpmaxT, RDT, tempC)

        sigma_Anet = pm.Uniform('sigma_Anet', lower=1e-6, upper=10)

        # Define likilyhood
        Anet = pm.Normal('Anet', mu=fA(Aj, Ac1), sigma=sigma_Anet, observed=obsAnet)

    # Sampling, numbers of draws and tune, target_accept are set based on study needs. Here I set 500 draws with 1500 tune as an example.
    # Draw: The number of samples to be drawn from the posterior distribution after the tuning phase.
    # Tune: The number of samples used for tuning the MCMC algorithm, which are discarded and not used for inference.
    # Target_accept: The target acceptance rate for the proposed samples in the MCMC algorithm.

    idata = pm.sample(model=model_1, draws=5000, tune=15000, target_accept=0.9)

    # Extract and save the draw data from four MCMC sampling chains
    posterior_samples = idata.posterior
    posterior_df = posterior_samples.to_dataframe()

    # Indicating which draw data are from which sampling chain. We have four chains running in the MCMC sampling, the chain index starts from 0
    posterior_df.reset_index(inplace=True)
    posterior_df.rename(columns={'chain': 'Chain'}, inplace=True)

    # Save samples to CSV file
    posterior_df.to_csv(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Sampled_idata_Exp_1/idata_{index_value}.csv", index=False)

    # Plot posterior
    plt.figure(figsize=(12, 8))
    az.plot_trace(idata)
    
    for ax in plt.gcf().axes:
        ax.title.set_fontsize(20)
        for label in ax.get_xticklabels()+ax.get_yticklabels():
            label.set_fontsize(18)

    plt.tight_layout()
    
    # Save plots
    plt.savefig(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Posterior_distribution_Exp_1/posterior_{index_value}.png", dpi=300)
    plt.close()

    # Save the summary of estiamted posterior distribution
    summary_df = az.summary(idata, stat_funcs={'median': np.median,'mode': mode})
    summary_df['Treat_index'] = index_value
    summary_df['Coefficient'] = ['alpha_PSII', 'Jmax', 'Rd', 'Vcmax', 'Vpmax', 'sigma_Anet']
    summary_df.to_csv(f"/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Alex_prior_sigma5/Estimated_coefficients_Exp_1/{index_value}.csv", index=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 280 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 212 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 315 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


Sampling 4 chains for 15_000 tune and 5_000 draw iterations (60_000 + 20_000 draws total) took 321 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_PSII, JmaxT, RdT, VcmaxT, VpmaxT, sigma_Anet]


...

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

Combine outputed .csv

In [3]:
# Open file where outputed .csv are. This need to be changed based on where the estimated posteriors saved
folder_path = '/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Uniformed_prior/Estimated_coefficients_Exp_1'

csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read .csv one by one
dfs = []
for file in csv_files:
    df1 = pd.read_csv(file)
    dfs.append(df1)

# Combine all DataFrame into one .csv
merged_outputs = pd.concat(dfs, ignore_index=True)

# Combine merged_outputs with Treat description
Treat_description = pd.read_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Treat_list_Exp_1.csv')

Fit_results = pd.merge(merged_outputs, Treat_description, on = 'Treat_index', how='left')

Fit_results.to_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Uniformed_prior/Estimated_coefficients_Exp_1/Combined_coefficients_Exp_1.csv', index=False) 

In [4]:
# Process the Treat column in the file, seperate it into Genotype, Date, Machine_type, Block, Leaf
Medium_process = pd.read_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Uniformed_prior/Estimated_coefficients_Exp_1/Combined_coefficients_Exp_1.csv', header=0)

Medium_process[['Genotype', 'Date', 'Machine_type', 'Block', 'Leaf']] = Medium_process['Treat'].str.split('_', expand=True)

Medium_process.to_csv('/Users/xiajin/Desktop/Photosyntheticdata/Exp.1/Bayesian_Photosynthesis_Exp_1_Uniformed_prior/Estimated_coefficients_Exp_1/Combined_coefficients_Exp_1_processed.csv', index=False)